Get the Cookies and headers More like a people.

In [14]:
from seleniumwire import webdriver  
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

# - step1 selenium 上網登入 ee-class
url = "https://w2.math.ncu.edu.tw/" # = ee-class url

# : user agent setting
ua = UserAgent()
user_agent = ua.random

# : option setting
chrome_opt = Options()
chrome_opt.add_argument("start-maximized")  # 設定瀏覽器啟動時開啟視窗最大化，提供較大的視野和操作空間
chrome_opt.add_argument("--disable-proxy-certificate-handler")  # 禁用代理伺服器的證書處理，避免因證書問題而影響網頁載入
chrome_opt.add_argument("--disable-content-security-policy")  # 禁用內容安全政策(CSP)，允許加載非同源的腳本，有助於測試開發
chrome_opt.add_argument("--ignore-certificate-errors")  # 忽略證書錯誤，有助於訪問自簽名證書的開發環境或測試站點
chrome_opt.add_argument("--incognito")  # 啟動無痕(隱身)模式，不會保存瀏覽歷史、Cookies等資料，有助於保護隱私和測試

# : driver setting 
driver = webdriver.Chrome(options = chrome_opt) # => insert the option to the driver (can be overridden)
driver.delete_all_cookies() # 防止污染
driver.get(url)
sleep(2)

# : driver get the title, content, links
driver_title = driver.title
driver_content = driver.find_element(By.TAG_NAME, "body").text
driver_all_links_elements = driver.find_elements(By.XPATH, "//a[@href]")

# : Get the Cookies & Headers
current_login_cookie = driver.get_cookies()
current_login_requests = driver.requests

headers = {}
for request in current_login_requests:
    if request.url == url:
        headers = {k: v for k, v in request.headers.items()}

print("Cookies:", current_login_cookie)
print("Headers:", headers)

sleep(5)
driver.quit()

Cookies: [{'domain': '.ncu.edu.tw', 'expiry': 1710658194, 'httpOnly': False, 'name': '_gat_gtag_UA_153670547_1', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '1'}, {'domain': 'w2.math.ncu.edu.tw', 'expiry': 1710665329, 'httpOnly': False, 'name': 'XSRF-TOKEN', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'eyJpdiI6IjVlWlpqWmRhbHZXS2ZoWDNvaDl4QlE9PSIsInZhbHVlIjoiUjQ5bGxLdm84S3pBZFlrUW16ZGs4VXJUZWZZdHFkZnVjc1ZxTEVLcFwvMFFaenlxcTZKNjNtTjljalwvN0JrdWtVIiwibWFjIjoiNjFkMTNkYjI2N2Y0ZjRiZjg4MWE5ZTE1MGI4MWFjNTZiZmMyZWRhZjQ5YThjNzkxN2JmZTA0YzVlMDY0ZDZkYiJ9'}, {'domain': '.ncu.edu.tw', 'expiry': 1745218134, 'httpOnly': False, 'name': '_ga', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GA1.3.1952377605.1710658134'}, {'domain': 'w2.math.ncu.edu.tw', 'expiry': 1710665329, 'httpOnly': True, 'name': 'laravel_session', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'eyJpdiI6ImFyWEJVK1l5eHVqaVNWQzJSdm96UXc9PSIsInZhbHVlIjoiNVwvN0kyVGJzR0xqbHlNNWp4WXZ1TVBZO

'國立中央大學數學系\n公告\n學術活動\n系所資訊\n系所成員\n課程介紹\n入學資訊\n活動紀錄\n立即捐款\n系PORTAL\n成績查詢\nLANGUAGE\n國立中央大學數學系\n數學科學組 計算與資料科學組\n置頂公告\n1122數學系研究生學位考試申請須知\n2024/02/21\n查看更多\n最新消息\n1122數學系研究生學位考試申請須知\n2024/02/21\n校慶系列講座第一場(2/27)：探索數學中的對稱之美\n2024/02/15\n校慶系列講座第三場(5/7)：「數學的魅力：從自然到科技」\n2024/02/15\n系友演講〖AI時代的求職力〗\n2023/11/15\n113學年度數學系碩士班甄試得參加複試名單及口試通知\n2023/10/27\n113學年度數學系特殊選才得參加複試名單\n2023/10/27\n113碩士班甄試9/27開始報名\n2023/09/26\n查看更多\n學術活動\nGaudin Hamiltonians on unitarizable modules over classical Lie (super)algebras\n2024/03/07\nGeometry and Topology in Mathematical AI\n2024/02/29\nTits alternative and its application\n2024/02/22\nEvolution of Diffusion Models: From Birth to Enhanced Efficiency and Controllability\n2024/02/22\nEarly Detection and Classification of Liver Fibrosis with Machine Learning Methods: Image-Based Biomarkers and Data-driven Computational Techniques in Dynamic Contrast-Enhanced MRI\n2024/01/09\nVariational Principles in Fluid Mechanics\n2023/12/21\nOn the newform theory for p-adic class

Request the website Content

In [5]:
import requests
from bs4 import BeautifulSoup
import re

try:
    with requests.Session() as session: 
    # ~ Session can update the performance of the server. 
    
        # : Update the Cookies & Headers from driver
        for cookie in current_login_cookie:
            session.cookies.set(cookie['name'], cookie['value'])
        session.headers.update(headers)
    
        # :  Get the response of the website.
        resp = session.get(url)
        
        # @ Check the response and Parser the response to HTML.
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        # @ Get the elements of the site.
        title = soup.title.get_text() if soup.title else ''
        
        content = soup.get_text().strip()
        content = re.sub(r'\n+', '\n', content)
        
        links = []
        all_links = soup.find_all(href=True)
        for link in all_links:
            href = link.get('href') 
            text = link.get_text().strip()

            if href:
                
                # : Get the Link and Href from the link
                if "http" not in href: 
                    href = f"{url}/{href}" if href[0] != "/" else f"{url}{href}" # = href that include the relative path
                
                if text == "": 
                    text = link.get('title','').strip()
                
                links.append((text,href))
                
    print(title)
    print(content)
            
except requests.HTTPError as e:
    print('HTTP error', e.response.status_code)
except requests.RequestException as e:
    print('Request error:', e)
except Exception as e:
    print('', '', [])

國立中央大學數學系
國立中央大學數學系
 
            國立中央大學數學系
        
公告
重要公告
最新消息
歷史公告
獎學金
學術活動
演講訊息
研討會公告
歷史紀錄
系所資訊
系所簡介
數學計算實驗室
機器人實驗室
圖書室
研究計畫
歷屆系主任
大事紀要
聯絡資訊
系友會
系所成員
專任教師
兼任教師
博士後研究員
訪問學者
行政人員
退休教師
課程介紹
修課規定
課程簡介
課程地圖
開放式課程
核心能力表
入學資訊
數學科學組
計算與資料科學組
學士班入學
碩士班入學
博士班入學
活動紀錄
演講紀錄
親師交流
學生活動
研討會紀錄
其他
立即捐款
系Portal
成績查詢
Language
English
Indonesian
Vietnamese
 國立中央大學數學系  
數學科學組
 計算與資料科學組
 置頂公告 
1122數學系研究生學位考試申請須知
2024/02/21
 查看更多 
 最新消息 
1122數學系研究生學位考試申請須知
2024/02/21
校慶系列講座第一場(2/27)：探索數學中的對稱之美
2024/02/15
校慶系列講座第三場(5/7)：「數學的魅力：從自然到科技」
2024/02/15
系友演講〖AI時代的求職力〗
2023/11/15
113學年度數學系碩士班甄試得參加複試名單及口試通知
2023/10/27
113學年度數學系特殊選才得參加複試名單
2023/10/27
113碩士班甄試9/27開始報名
2023/09/26
 查看更多 
 學術活動 
Gaudin Hamiltonians on unitarizable modules over classical Lie (super)algebras
2024/03/07
Geometry and Topology in Mathematical AI
2024/02/29
Tits alternative and its application
2024/02/22
Evolution of Diffusion Models: From Birth to Enhanced Efficiency and Controllability
2024/02/22
Early Detection and Classification of Liver 

In [9]:
with requests.Session() as session: 
# ~ Session can update the performance of the server. 

    # : Update the Cookies & Headers from driver
    for cookie in current_login_cookie:
        session.cookies.set(cookie['name'], cookie['value'])
    session.headers.update(headers)

    # :  Get the response of the website.
    resp = session.get(url)
    
    # @ Check the response and Parser the response to HTML.
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # @ Get the elements of the site.
    title = soup.title.get_text() if soup.title else ''
    
    content = soup.get_text().strip()
    content = re.sub(r'\n+', '\n', content)
    
content

'國立中央大學數學系\n \n            國立中央大學數學系\n        \n公告\n重要公告\n最新消息\n歷史公告\n獎學金\n學術活動\n演講訊息\n研討會公告\n歷史紀錄\n系所資訊\n系所簡介\n數學計算實驗室\n機器人實驗室\n圖書室\n研究計畫\n歷屆系主任\n大事紀要\n聯絡資訊\n系友會\n系所成員\n專任教師\n兼任教師\n博士後研究員\n訪問學者\n行政人員\n退休教師\n課程介紹\n修課規定\n課程簡介\n課程地圖\n開放式課程\n核心能力表\n入學資訊\n數學科學組\n計算與資料科學組\n學士班入學\n碩士班入學\n博士班入學\n活動紀錄\n演講紀錄\n親師交流\n學生活動\n研討會紀錄\n其他\n立即捐款\n系Portal\n成績查詢\nLanguage\nEnglish\nIndonesian\nVietnamese\n 國立中央大學數學系  \n數學科學組\n 計算與資料科學組\n 置頂公告 \n1122數學系研究生學位考試申請須知\n2024/02/21\n 查看更多 \n 最新消息 \n1122數學系研究生學位考試申請須知\n2024/02/21\n校慶系列講座第一場(2/27)：探索數學中的對稱之美\n2024/02/15\n校慶系列講座第三場(5/7)：「數學的魅力：從自然到科技」\n2024/02/15\n系友演講〖AI時代的求職力〗\n2023/11/15\n113學年度數學系碩士班甄試得參加複試名單及口試通知\n2023/10/27\n113學年度數學系特殊選才得參加複試名單\n2023/10/27\n113碩士班甄試9/27開始報名\n2023/09/26\n 查看更多 \n 學術活動 \nGaudin Hamiltonians on unitarizable modules over classical Lie (super)algebras\n2024/03/07\nGeometry and Topology in Mathematical AI\n2024/02/29\nTits alternative and its application\n2024/02/22\nEvolution of Diffusion Models: From Birth to Enhanced Efficiency

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 啟動 Chrome 瀏覽器。
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 訪問指定的 URL。
driver.get("https://w2.math.ncu.edu.tw//member/full")

# 等待一段時間，讓網頁完全加載。
time.sleep(5)  # 等待 5 秒，根據實際情況調整。

# 獲取網頁標題和渲染後的內容。
title = driver.title
content = driver.find_element(By.TAG_NAME, "body").text

sleep(2)

# 獲取網頁上所有的連結
all_links_elements = driver.find_elements(By.XPATH, "//a[@href]")

links = []
for element in all_links_elements:
    href = element.get_attribute('href')
    text = element.text.strip()

    if href:
        # 確保連結是完整的 URL
        if "http" not in href:
            href = f"{url}/{href}" if href[0] != "/" else f"{url}{href}"

        if text == "":
            # 如果連結文本為空，嘗試獲取 title 屬性
            text = element.get_attribute('title').strip()

        links.append((text, href))

# 顯示所有連結
for text, href in links:
    print(f"Text: {text}, Href: {href}")

print(title)
print(content)

# 關閉瀏覽器。
driver.quit()

Text: 國立中央大學數學系, Href: https://w2.math.ncu.edu.tw/
Text: 公告, Href: https://w2.math.ncu.edu.tw//member/full#
Text: , Href: https://w2.math.ncu.edu.tw/annou/top
Text: , Href: https://w2.math.ncu.edu.tw/annou/news
Text: , Href: https://w2.math.ncu.edu.tw/annou/history
Text: , Href: https://w2.math.ncu.edu.tw/annou/scholarship
Text: 學術活動, Href: https://w2.math.ncu.edu.tw//member/full#
Text: , Href: https://w2.math.ncu.edu.tw/academic/speeches
Text: , Href: https://w2.math.ncu.edu.tw/academic/seminars
Text: , Href: https://w2.math.ncu.edu.tw/academic/history
Text: 系所資訊, Href: https://w2.math.ncu.edu.tw//member/full#
Text: , Href: https://w2.math.ncu.edu.tw/about/info
Text: , Href: https://w2.math.ncu.edu.tw/about/mcl
Text: , Href: https://w2.math.ncu.edu.tw/about/aimakerlab
Text: , Href: https://w2.math.ncu.edu.tw/about/library
Text: , Href: https://w2.math.ncu.edu.tw/about/plan
Text: , Href: https://w2.math.ncu.edu.tw/about/chairman
Text: , Href: https://w2.math.ncu.edu.tw/about/event
Text

In [21]:
for element in all_links_elements:
    href = element.get_attribute('href')

MaxRetryError: HTTPConnectionPool(host='localhost', port=51330): Max retries exceeded with url: /session/b7d5bd547c114e16d9bb3bb6231a991f/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x107b31610>: Failed to establish a new connection: [Errno 61] Connection refused'))